In [47]:
import pandas as pd
import matplotlib.pyplot as plt

# Load DATA

In [48]:

gdpdf = pd.read_csv('https://raw.githubusercontent.com/boardkeystown/CIS568Project/main/Data/API_NY.GDP.MKTP.KD.ZG_DS2_en_csv_v2_4694019.csv')

avg_height_men = pd.read_csv("https://raw.githubusercontent.com/boardkeystown/CIS568Project/main/Data/average-height-of-men.csv")
avg_height_women = pd.read_csv("https://raw.githubusercontent.com/boardkeystown/CIS568Project/main/Data/average-height-of-women.csv")

avg_height_change_women = pd.read_csv('https://raw.githubusercontent.com/boardkeystown/CIS568Project/main/Data/annual-change-in-average-female-height.csv')
avg_height_change_men = pd.read_csv('https://raw.githubusercontent.com/boardkeystown/CIS568Project/main/Data/annual-change-in-average-male-height.csv')

Filter out na values, set to 0

In [49]:
gdpdf = gdpdf.fillna(0)
avg_height_men = avg_height_men.fillna(0)
avg_height_women = avg_height_women.fillna(0)
avg_height_change_women = avg_height_change_women.fillna(0)
avg_height_change_men  = avg_height_change_men.fillna(0)

# Get Data We Want To Use Into Single Dataframe

In [50]:
#copy men into new df and start with this
avg_height_human = avg_height_men.copy()

In [51]:
avg_height_human.head()

,Entity,Code,Year,Mean male height (cm)
0,Afghanistan,AFG,1896,161.164095
1,Afghanistan,AFG,1897,161.196286
2,Afghanistan,AFG,1898,161.228297
3,Afghanistan,AFG,1899,161.260727
4,Afghanistan,AFG,1900,161.293068


In [52]:
#copy over other columns

avg_height_human['Mean female height (cm)'] = avg_height_women['Mean female height (cm)'][(avg_height_human['Code'] == avg_height_women['Code']) & (avg_height_human['Year'] == avg_height_women['Year'])]
avg_height_human['change in male height'] = avg_height_change_men['Year-on-year change in male height (%)']
avg_height_human['change in female height'] = avg_height_change_women['Year-on-year change in female height (%)']


#do fill na to be safe

avg_height_human = avg_height_human.fillna(0)
avg_height_human.head()


,Entity,Code,Year,Mean male height (cm),Mean female height (cm),change in male height,change in female height
0,Afghanistan,AFG,1896,161.164095,149.187747,0.01997,0.08962
1,Afghanistan,AFG,1897,161.196286,149.321451,0.01986,0.08977
2,Afghanistan,AFG,1898,161.228297,149.455494,0.02011,0.08966
3,Afghanistan,AFG,1899,161.260727,149.589503,0.02005,0.08963
4,Afghanistan,AFG,1900,161.293068,149.723587,0.02010,0.08949


In [53]:
#take a look at GDP 

gdpdf.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.035375,6.429817,-0.020479,3.543630,2.116716,5.482371,1.267218,-2.085600,-22.318948,0.000000
1,Africa Eastern and Southern,AFE,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,0.0,0.239594,7.980768,5.164334,4.579952,5.327717,...,0.923067,4.200209,3.984387,2.952845,2.217017,2.557135,2.485244,2.025941,-2.886856,4.303667
2,Afghanistan,AFG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,12.752287,5.600745,2.724543,1.451315,2.260314,2.647003,1.189228,3.911603,-2.351101,0.000000
3,Africa Western and Central,AFW,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,0.0,1.848752,3.770220,7.272440,5.396349,4.049793,...,5.142997,6.104192,5.927355,2.745894,0.127844,2.317679,2.952190,3.202544,-0.897014,3.914799
4,Angola,AGO,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.540000,4.950000,4.820000,0.940000,-2.580000,-0.150000,-1.316312,-0.700000,-5.500000,0.700000


Make a few functions to filter the GDP columns

In [54]:
#need to filter by columns into the avg_height_human df 
#get the years columns indexes
def filterGDPYears(df):
    gdpYears = df.columns[5:len(df.columns)-1]
    gdpYearsLookUp =  {}
    indexList = []
    yearsList = []
    for i,j in enumerate(gdpYears):
        gdpYearsLookUp[j] = i+5
        indexList.append(i+5)
        yearsList.append(j)
    return gdpYears, gdpYearsLookUp, indexList, yearsList


gdpYears, gdpYearsLookUp, indexList, yearsList = filterGDPYears(gdpdf)


In [55]:
import math

def getGDP_DFMap(df,yList,iList):
    map = {}
    for row in gdpdf.iterrows():
        if (map.get(row[1]['Country Code'],False) is False):
            s = {}
            for i,j in enumerate(iList):
                v = 0
                if not math.isnan(row[1][j]):
                    v = row[1][j]
                s |= {f"{yList[i]}":v}
            map[row[1]['Country Code']] = s
    return map


gdpdfMap = getGDP_DFMap(gdpdf,yearsList,indexList)

Now we have a way to get the info we are looking for from the gdp datasets

In [56]:
gdpdfMap.keys()

dict_keys(['ABW', 'AFE', 'AFG', 'AFW', 'AGO', 'ALB', 'AND', 'ARB', 'ARE', 'ARG', 'ARM', 'ASM', 'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CEB', 'CHE', 'CHI', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CSS', 'CUB', 'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'EAP', 'EAR', 'EAS', 'ECA', 'ECS', 'ECU', 'EGY', 'EMU', 'ERI', 'ESP', 'EST', 'ETH', 'EUU', 'FCS', 'FIN', 'FJI', 'FRA', 'FRO', 'FSM', 'GAB', 'GBR', 'GEO', 'GHA', 'GIB', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 'GUM', 'GUY', 'HIC', 'HKG', 'HND', 'HPC', 'HRV', 'HTI', 'HUN', 'IBD', 'IBT', 'IDA', 'IDB', 'IDN', 'IDX', 'IMN', 'IND', 'INX', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KIR', 'KNA', 'KOR', 'KWT', 'LAC', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LCN', 'LDC', 'LIC', 'LIE', 'LKA', 'LMC', 'LMY', 'L

In [57]:
gdpdfMap["JPN"]

{'1961': 12.04353641,
 '1962': 8.908972996,
 '1963': 8.473642383,
 '1964': 11.6767082,
 '1965': 5.819707873,
 '1966': 10.63856156,
 '1967': 11.08214232,
 '1968': 12.88246814,
 '1969': 12.47789451,
 '1970': 2.454958384,
 '1971': 4.698992042,
 '1972': 8.413547255,
 '1973': 8.032599975,
 '1974': -1.225239828,
 '1975': 3.091575916,
 '1976': 3.974984091,
 '1977': 4.39033795,
 '1978': 5.271941503,
 '1979': 5.484041832,
 '1980': 2.817591208,
 '1981': 4.260624493,
 '1982': 3.279742743,
 '1983': 3.630198572,
 '1984': 4.410880154,
 '1985': 5.159807905,
 '1986': 3.294047613,
 '1987': 4.648861398,
 '1988': 6.661636522,
 '1989': 4.925785413,
 '1990': 4.840929057,
 '1991': 3.523357235,
 '1992': 0.900586066,
 '1993': -0.459219716,
 '1994': 1.083383174,
 '1995': 2.630999616,
 '1996': 3.133870993,
 '1997': 0.981228732,
 '1998': -1.270330495,
 '1999': -0.333929958,
 '2000': 2.764647551,
 '2001': 0.386103426,
 '2002': 0.041962499,
 '2003': 1.535125499,
 '2004': 2.186115694,
 '2005': 1.803900872,
 '2006':

Need a list of country codes

In [58]:
ccode = gdpdf['Country Code']
ccode

0      ABW
1      AFE
2      AFG
3      AFW
4      AGO
      ... 
261    XKX
262    YEM
263    ZAF
264    ZMB
265    ZWE
Name: Country Code, Length: 266, dtype: object

Now need to figure out what was in the map so that we can add to each row of the avg_height_human df if we have the data for it.

In [59]:
def makeGDPColumn(df,map):
    columnGDPKeep = []
    for row in avg_height_human.iterrows():
        v = 0
        if (map.get(code := row[1]['Code'],False)):
            if (map[code].get(year := str(row[1]['Year']),False)):
                v = map[code][year]
        columnGDPKeep.append(v)
    return columnGDPKeep

GDPKeep = makeGDPColumn(avg_height_human,gdpdfMap)

In [60]:
avg_height_human["GDP annual growth"] = GDPKeep
avg_height_human.head(3)

,Entity,Code,Year,Mean male height (cm),Mean female height (cm),change in male height,change in female height,GDP annual growth
0,Afghanistan,AFG,1896,161.164095,149.187747,0.01997,0.08962,0.0
1,Afghanistan,AFG,1897,161.196286,149.321451,0.01986,0.08977,0.0
2,Afghanistan,AFG,1898,161.228297,149.455494,0.02011,0.08966,0.0


In [61]:
avg_height_human.tail(3)

,Entity,Code,Year,Mean male height (cm),Mean female height (cm),change in male height,change in female height,GDP annual growth
21005,Zimbabwe,ZWE,1994,168.963595,158.400025,0.0,0.0,9.235199
21006,Zimbabwe,ZWE,1995,168.774934,158.307779,0.0,0.0,0.158026
21007,Zimbabwe,ZWE,1996,168.586005,158.215186,0.0,0.0,10.360697


In [62]:

print(f"         gdp years: max - min:{gdpYears.max()},{gdpYears.min()}")
print(f"height_human years: max - min:{avg_height_human['Year'].max()},{avg_height_human['Year'].min()}")



         gdp years: max - min:2020,1961
height_human years: max - min:1996,1896


The human height index dataset has years then we have gdp data for. So we should remove years we do not have.

In [63]:
dropDf = avg_height_human[(avg_height_human['Year'] < 1960)].index
avg_height_human.drop(dropDf , inplace=True)

In [64]:
avg_height_human.head(3)

,Entity,Code,Year,Mean male height (cm),Mean female height (cm),change in male height,change in female height,GDP annual growth
64,Afghanistan,AFG,1960,164.742777,156.404249,0.03299,0.02085,0.0
65,Afghanistan,AFG,1961,164.797132,156.436857,0.03198,0.01791,0.0
66,Afghanistan,AFG,1962,164.849830,156.464874,0.02991,0.01378,0.0


NOW LETS SAVE THE DATASET TO CSV

In [65]:
avg_height_human.to_csv("avg_height_human_country_gdp.csv", index=False)

# Lets look at the data using python

In [66]:
import numpy as np  ## Numpy is one of the default data science libraries, it contains numerous functions that help analyze and reshaping data
# Matplotlib
import matplotlib.pyplot as plt ## matplotlib is the default visualization library for python 